<a href="https://colab.research.google.com/github/10100111/Display-of-HW1/blob/main/82_Ultra_light_%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D1%8F_%D0%BC%D1%83%D0%B7%D1%8B%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание Ultra Lite

Макс 10 баллов

Перепишите этапы подготовки данных и обучения модели с нуля. Для лучшего усвоения материала постарайтесь самостоятельно печатать код, даже если Вы будете его подсматривать в блокноте с занятия.

# Загрузка библиотек

In [ ]:
import pandas as pd
import numpy as np
import keras
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.callbacks import ModelCheckpoint
from google.colab import files, drive
from matplotlib import pyplot as plt
from music21 import *
import glob
import pickle
from IPython.display import clear_output
from IPython.display import Audio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Загрузка данных

In [ ]:
# Загрузка и unzip 
!unzip '/content/drive/MyDrive/Базы/class.zip' -d './data/'
clear_output()

In [ ]:
# Функция парсит midi файл в удобный для работы формат
midi = converter.parse('/content/data/alb_esp1.mid')
midi

<music21.stream.Score 0x7fac04b603a0>

In [ ]:
# Функция получения списка нот и аккордов в файле
notes_to_parse = midi.flat.notes
for element in notes_to_parse[:40]:
  print(element, element.offset) # offset - это смещение, которое позволяет проигрывать ноту в разное время
  

<music21.note.Note A> 0.5
<music21.note.Note A> 0.5
<music21.note.Note E> 1.0
<music21.note.Note E> 1.0
<music21.note.Note D> 4.25
<music21.note.Note D> 4.25
<music21.note.Note E> 13/3
<music21.note.Note E> 13/3
<music21.note.Note D> 4.5
<music21.note.Note D> 4.5
<music21.note.Note C> 5.0
<music21.note.Note C> 5.0
<music21.note.Note D> 5.5
<music21.note.Note D> 5.5
<music21.note.Note E> 6.0
<music21.note.Note E> 6.0
<music21.note.Note F> 6.5
<music21.note.Note F> 6.5
<music21.note.Note G> 7.0
<music21.note.Note G> 7.0
<music21.note.Note E> 7.5
<music21.note.Note E> 7.5
<music21.note.Note F> 8.0
<music21.note.Note F> 8.0
<music21.note.Note E> 8.5
<music21.note.Note E> 8.5
<music21.note.Note D> 8.75
<music21.note.Note E> 8.75
<music21.note.Note D> 8.75
<music21.note.Note E> 8.75
<music21.note.Note D> 9.0
<music21.note.Note D> 9.0
<music21.note.Note C> 10.0
<music21.note.Note C> 10.0
<music21.note.Note B-> 11.0
<music21.note.Note B-> 11.0
<music21.note.Note A> 12.0
<music21.note.Note A> 1

In [ ]:
# Выбираем пианино, если в файле есть несколько инструментов
s2 = instrument.partitionByInstrument(midi)
s2.parts[0] # берем часть с пианино

<music21.stream.Part Piano>

# Собираем список нот

In [ ]:
notes = [] # здесь будут храниться все ноты из всех файлов
for i, file in enumerate(glob.glob('/content/data/*.mid')):
  midi = converter.parse(file)       # парсит midi файл в удобный для работы формат
  print('\r', 'Обрабатывается файл', i, ' ', file, end='')
  notes_to_parse = None
  try:                               # если в файле нет разных инструментов
    notes_to_parse = midi.flat.notes # получаем списки нот и аккордов
  except:                            # если есть разные инструменты
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse() # итератор


  for element in notes_to_parse:          # добавляем в список ноту или аккорд
    if isinstance(element, note.Note): # если нота, то росто добавляем ее
      notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord): # если аккорд, то записываем числовой вариант всех нот через точку
      notes.append('.'.join(str(n) for n in element.normalOrder))



 Обрабатывается файл 80   /content/data/chpn-p3.mid

In [ ]:
notes_ = []
midi = converter.parse('/content/data/alb_esp1.mid')
s2 = instrument.partitionByInstrument(midi)
notes_to_parse = s2.parts[0].recurse() # итератор
for element in notes_to_parse: # добавляем в список ноту или аккорд 
  # if isinstance(element, note.Note): # если нота, просто добавляем ее
  #   notes_.append(str(element.pitch))
  if isinstance(element, chord.Chord): # если аккорд, то записываем числовой вариант всех нот через точку
    notes_.append('.'.join(str(n) for n in element.normalOrder))

In [ ]:
notes_[:10]

['1.4', '10.2.4', '1.4', '10.2.4', '4.10', '2.8', '4.9', '9.1.4']

# Сохраняем полученный список на диск и загружаем

In [ ]:
# Сохранение
with open('notes', 'wb') as filepath:
  pickle.dump(notes, filepath)

In [ ]:
# Загрузка
with open('/content/notes', 'rb') as file:
  notes = pickle.load(file)

# Подготовка данных

In [ ]:
notes[510:520]

['G3', 'E-4', 'C#4', 'B-4', 'G3', 'C#5', 'E-3', 'G5', 'G#2', 'B-4']

In [ ]:
len(notes)

139269

In [ ]:
# Применим метод set чтобы собрать уникальные ноты и аккорды
n_list = set(notes)
# Выведем их количество 
len(n_list)

351

In [ ]:
# Зададим длину последовательности нот/аккордов по которой сеть будет делать предсказания
sequence_length = 100

# Получаем отсортированный список всех уникальных нот и аккордов
pitchnames = sorted(n_list)

# Создаем словарь, сопоставляющий ноту/аккорд числу (int)
note_to_int = dict((note, number) for number, note in enumerate(pitchnames)) 

network_input = [] # список со всеми последовательностями
network_output = [] # список со всеми элементами, идущими после последовательности

# Создаем обучающую выборку
for i in range(0, len(notes) - sequence_length, 1): # проходимся окном по всем элементам за минусом "хвоста" = окну
  sequence_in = notes[i:i + sequence_length] # выбираем (окно) последовательность длины sequence_length
  sequence_out = notes[i + sequence_length] # следующий за последовательностью элемент
  network_input.append([note_to_int[char] for char in sequence_in]) # добавляем в список последовательность, предварительно, переведя все ноты/аккорды в числа
  network_output.append(note_to_int[sequence_out]) # добавляем в список следующий за последовательностью элемент, переведя его в число

n_patterns = len(network_input) # количество полученных последовательностей

# Делаем решейп, чтобы сеть могла обучаться
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# Нормализуем входные данные
network_input = network_input / float(len(n_list))
# Выходные данные преобразыем в OHE
network_output = np.array(network_output)
# network_output = keras.utils.to_categorical(network_output)

In [ ]:
network_input.shape

(139169, 100, 1)

# Архитектура сети

In [ ]:
def create_network(network_input, n_list, dropout=0.3):
  model = Sequential()
  model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
  model.add(Dropout(dropout))
  model.add(LSTM(256, return_sequences=True))
  model.add(Dropout(dropout))
  model.add(LSTM(256))
  model.add(Dense(128))
  model.add(Dropout(dropout))
  model.add(Dense(len(n_list)))
  model.add(Activation('softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

  return model

In [ ]:
model = create_network(network_input, n_list)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 256)          264192    
                                                                 
 dropout (Dropout)           (None, 100, 256)          0         
                                                                 
 lstm_1 (LSTM)               (None, 100, 256)          525312    
                                                                 
 dropout_1 (Dropout)         (None, 100, 256)          0         
                                                                 
 lstm_2 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0

In [ ]:
'''
Для обучения сети используем ModelCheckpoint, 
позволяющий сохранять веса в процессе обучения
'''
# Пропишем путь к сохранению весов
filepath = '/content/drive/MyDrive/Базы/Генерация_музыки/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5'
checkpoint = ModelCheckpoint(filepath,
                             monitor='loss',
                             verbose=0,
                             save_weights_only=True, # сохраняем только веса
                             save_best_only=True, # сохраняем только при уменьшении ошибки
                             mode='min',     
                             )
callbacks_list = [checkpoint]

model.fit(network_input, 
          network_output, 
          epochs=1, 
          batch_size=128, 
          callbacks=callbacks_list
          )

1088/1088 [==============================] - 755s 689ms/step - loss: 4.6390 - accuracy: 0.0237


1088/1088 [==============================] - 775s 712ms/step - loss: 4.5732 - accuracy: 0.0258


In [ ]:
# Загружаем веса из сохраненного файла на гугл диске
weights = '/content/drive/MyDrive/Базы/Генерация_музыки/weights-improvement-01-4.3511.hdf5'
if (len(weights)>0):
  model.load_weights(weights)


In [ ]:
# Дообучим сеть и назначим новую ссылку для сохранения весов improvement_1
filepath = '/content/drive/MyDrive/Базы/Генерация_музыки/weights-improvement_1-{epoch:02d}-{loss:.4f}.hdf5'
checkpoint = ModelCheckpoint(filepath,
                             monitor='loss',
                             verbose=0,
                             save_weights_only=True,
                             save_best_only=True,
                             mode='min',
                             )
callback_list = [checkpoint]

model.fit(network_input,
          network_output,
          epochs=1,
          batch_size=128,
          callbacks=callback_list)

1088/1088 [==============================] - 772s 710ms/step - loss: 4.3023 - accuracy: 0.0613


In [ ]:
weights = '/content/drive/MyDrive/Базы/Генерация_музыки/weights-improvement_1-01-4.3023.hdf5'
if (len(weights)>0):
  model.load_weights(weights)

In [ ]:
model.fit(network_input,
          network_output,
          epochs=1,
          batch_size=128)

1088/1088 [==============================] - 773s 711ms/step - loss: 4.2263 - accuracy: 0.0720


# Генерация музыки

Для генерации музыки будем использовать следующий подход:

1. Выбираем случайную последовательность из обучающей выборки (далее паттерн)

2. Передаем паттерн модели для генерации нового элемента

3. Добавляем полученный элемент в финальный список и в конец паттерна

4. Удаляем первый элемент в паттерне

5. Возвращаемся к шагу 2